In [ ]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from datetime import datetime, timedelta, date
import datetime
import vertica_python
import io
from tqdm import tqdm
from clickhouse_driver import Client
import xlsxwriter

#connect to Clickhouse

client = Client('clickhouse',
               user = login,
               password = password,
               database = database)

settings = {'max_rows_to_group_by':1000000000000000000000000,
           'max_execution_time':13000000000000000000000000,
           'max_result_rows' :1000000000000000000000000
           }

In [ ]:
#generating range between date

date_start = '2022-12-29'
date_end = '2023-01-13'

date_startt = datetime.datetime.strptime(date_start, '%Y-%m-%d')
date_endd = datetime.datetime.strptime(date_end, '%Y-%m-%d')

daterange = [(date_startt + timedelta(days=x)).strftime('%Y-%m-%d') for x in range((date_endd-date_startt).days)]

#brand id

brand_id = 0000000

#sku

sku = (0000000000)

#highlight ID

highlight_id = 0000000000000

print(f'Period: from {date_start} to {date_end}')
print(f'Brand ID: {brand_id}')
print(f'Sku list: {sku}')

In [ ]:
df = pd.DataFrame()
for i in tqdm(daterange):
    stats_query = f"""SELECT date                                                                               as Date,
                   page_highlight_id                                                                  as HighlightID,
                   countIf(action_type, action_type = 'page_view') as Views,
                   uniqCombinedIf(user_session_id, action_type = 'page_view')                         as UsersUniq,
                   countIf(action_type,action_type = 'click' and object_type = 'product') as Clicks,
                   uniqCombinedIf(user_session_id, action_type = 'click' and object_type = 'product') as ClicksUniq
            FROM tracker.events
            WHERE date = '{i}'
              --AND highlight_id != ''
              AND HighlightID > 0
              and HighlightID in ({highlight_id})
              AND attributes_namespace = 'bx'
            GROUP BY date, page_highlight_id;"""
    
    result = pd.DataFrame(client.execute(stats_query,settings = settings)).rename(columns = {0:'Date', 1:'HighlightID', 2:'Views', 3:'UsersUniq', 4:'Clicks', 5:'ClicksUniq'} )
    
    df = pd.concat([df,result])
df

In [ ]:
df1 = pd.DataFrame()
for x in tqdm(daterange):
    stats2_query = f"""SELECT date                                                     as Date,
                   if(page_highlight_id < 4,
                      toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')), page_highlight_id)                                    as HighlightID,
                   countIf(action_type, action_type = 'to_cart')            as AddtoCart,
                   uniqCombinedIf(user_session_id, action_type = 'to_cart') as AddtoCartUniq
            FROM tracker.events
            WHERE date = '{x}'
              -- AND highlight_id != ''
              AND HighlightID > 0
              and HighlightID in ({highlight_id})
              AND action_type = 'to_cart'
              AND object_type = 'product'
              AND object_sku != 0
            GROUP BY date,
                     if(page_highlight_id < 4,
                        toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')),
                        page_highlight_id);"""
    
    
    result = pd.DataFrame(client.execute(stats2_query,settings = settings)).rename(columns = {0:'Date', 1:'HighlightID', 2:'AddtoCart', 3:'AddtoCartUniq'} )
    
    df1 = pd.concat([df1,result])
df1

In [ ]:
df2 = pd.DataFrame()
for z in tqdm(daterange):
    stats3_query = f"""SELECT date                          as Date,
                   highlight_id                  as HighlightID,
                   uniqCombined(user_session_id) as Clients,
                   uniqCombined(object_order_id) as Orders,
                   sum(quantity)                 as Qty,
                   SUM(quantity * finalprice)    as GMV
            FROM (
                  SELECT date,
                         attributes_platform,
                         highlight_id,
                         user_session_id,
                         object_sku,
                         sku_in_order,
                         t2.object_order_id,
                         quantity,
                         finalprice
                  FROM (SELECT date,
                               attributes_platform,
                               if(
                                           page_highlight_id < 4,
                                           toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')),
                                           page_highlight_id
                                   )                                                            as highlight_id,
                               object_sku,
                               user_session_id,
                               min(server_timestamp)                                            as min_server_timestamp
                        FROM tracker.events
                        WHERE date = '{z}'
                          --AND highlight_id != ''
                          AND highlight_id > 0
                          AND action_type in ('to_cart', 'favorite')
                          AND object_type = 'product'
                          AND object_sku != 0
                        group by date, attributes_platform,
                                 if(
                                             page_highlight_id < 4,
                                             toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')),
                                             page_highlight_id
                                     ), object_sku, user_session_id) as t1
                           INNER JOIN (
                      SELECT user_session_id
                           , object_order_id
                           , sku_in_order
                           , quantity
                           , finalprice
                           , min(server_timestamp) as min_server_timestamp
                      FROM tracker.events
                          ARRAY JOIN
                           properties_items.sku as sku_in_order,
                           properties_items.quantity as quantity,
                           properties_items.final_price as finalprice
                      WHERE date BETWEEN '{z}' AND toDate('{z}') + 1
                        and attributes_namespace = 'bx'
                        and action_type = 'payment_success'
                      group by user_session_id
                             , object_order_id
                             , sku_in_order
                             , quantity, finalprice) as t2
                                      ON t1.object_sku = t2.sku_in_order and t1.user_session_id = t2.user_session_id
                  WHERE object_order_id != ''
                    and t1.min_server_timestamp < t2.min_server_timestamp)
            WHERE highlight_id in ({highlight_id})
            GROUP BY date, highlight_id;"""
    
    
    result = pd.DataFrame(client.execute(stats3_query,settings = settings)).rename(columns = {0:'Date', 1:'HighlightID', 2:'Clients', 3:'Orders', 4:'Qty', 5:'GMV'} )
    
    df2 = pd.concat([df2,result])
df2

In [ ]:
df3 = pd.DataFrame()
for c in tqdm(daterange):
    stats4_qeury = f"""SELECT date                          as Date,
                   highlight_id                  as HighlightID,
                   MiniBU,
                   uniqCombined(user_session_id) as Clients,
                   uniqCombined(object_order_id) as Orders,
                   sum(quantity)                 as Qty,
                   SUM(quantity * finalprice)    as GMV
            FROM (
                  SELECT date,
                         attributes_platform,
                         highlight_id,
                         CASE
                            WHEN MiniBU in ('PetCare', 'BabyFood', 'DryFood', 'Beverages', 'Food', 'Fabric&Home', 'Cotton&Paper', 'Hygiene', 'Pharmacy', 'Kids Goods&Sport', 'Toys', 'School&Stationery&Equipment', 'Hobbies&Creativity')
                                 THEN MiniBU
                             ELSE 'Other' END as MiniBU,
                         user_session_id,
                         object_sku,
                         sku_in_order,
                         t2.object_order_id,
                         quantity,
                         finalprice
                  FROM (SELECT date,
                               attributes_platform,
                               if(
                                           page_highlight_id < 4,
                                           toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')),
                                           page_highlight_id
                                   )                                                            as highlight_id,
                               object_sku,
                               dictGetString('commercial_category', 'name', (arrayElement(dictGetHierarchy('commercial_category',
                                                                                                           dictGet('sku', 'commercial_category_id', toUInt64(object_sku))),
                                                                                          -1))) as MiniBU,
                               user_session_id,
                               min(server_timestamp)                                            as min_server_timestamp
                        FROM tracker.events
                        WHERE date = '{c}'
                          --AND highlight_id != ''
                          AND highlight_id > 0
                          AND action_type in ('to_cart', 'favorite')
                          AND object_type = 'product'
                          AND object_sku != 0
                        group by date, attributes_platform,
                                 if(
                                             page_highlight_id < 4,
                                             toUInt32OrNull(extract(page_referral_url, 'highlight.*?([0-9]{{4,8}})')),
                                             page_highlight_id
                                     ), object_sku, user_session_id) as t1
                           INNER JOIN (
                      SELECT user_session_id
                           , object_order_id
                           , sku_in_order
                           , quantity
                           , finalprice
                           , min(server_timestamp) as min_server_timestamp
                      FROM tracker.events
                          ARRAY JOIN
                           properties_items.sku as sku_in_order,
                           properties_items.quantity as quantity,
                           properties_items.final_price as finalprice
                      WHERE date = '{c}'
                        and attributes_namespace = 'bx'
                        and action_type = 'payment_success'
                      group by user_session_id
                             , object_order_id
                             , sku_in_order
                             , quantity, finalprice) as t2
                                      ON t1.object_sku = t2.sku_in_order and t1.user_session_id = t2.user_session_id
                  WHERE object_order_id != ''
                    and t1.min_server_timestamp < t2.min_server_timestamp)
            WHERE highlight_id in ({highlight_id})
            GROUP BY date, highlight_id, MiniBU;"""
    
    
    result = pd.DataFrame(client.execute(stats4_qeury,settings = settings)).rename(columns = {0:'Date', 1:'HighlightID', 2:'MiniBU', 3:'Clients', 4:'Orders', 5:'Qty', 6:'GMV'} )
    
    df3 = pd.concat([df3,result])   
df3

In [ ]:
join1 = df.join(df1, how = 'outer', on = 'Date' )
join2 = join1.join(df2, how = 'outer', on = 'Date' )
join3 = join2.join(df3, how = 'outer', on = 'Date')
join3.head()

In [ ]:
writer = pd.ExcelWriter('PF-2385.xlsx', engine='xlsxwriter')
join3.to_excel(writer, sheet_name='stats')
writer.close()